In [1]:
!pip install kaggle
!pip install permetrics

     |████████████████████████████████| 118 kB 4.4 MB/s eta 0:00:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.10
    Uninstalling urllib3-1.25.10:
      Successfully uninstalled urllib3-1.25.10
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

pyppeteer 0.2.2 requires urllib3<2.0.0,>=1.25.8, but you'll have urllib3 1.24.3 which is incompatible.
influxdb 5.3.0 requires msgpack==0.6.1, but you'll have msgpack 1.0.0 which is incompatible.
     |████████████████████████████████| 63 kB 1.3 MB/s eta 0:00:011


In [2]:
! mkdir ~/.kaggle

In [3]:
! cp kaggle.json ~/.kaggle/

In [4]:
!kaggle competitions download -c m5-forecasting-accuracy

 98%|█████████████████████████████████████▎| 45.0M/45.8M [00:02<00:00, 20.6MB/s]
100%|██████████████████████████████████████| 45.8M/45.8M [00:02<00:00, 18.4MB/s]


In [5]:
!unzip m5-forecasting-accuracy.zip

Archive:  m5-forecasting-accuracy.zip
  inflating: calendar.csv            
  inflating: sales_train_evaluation.csv  
  inflating: sales_train_validation.csv  
  inflating: sample_submission.csv   
  inflating: sell_prices.csv         


In [1]:
import pandas as pd
import numpy as np
from utils import *

In [2]:
path_m5 = "./"
calendar = pd.read_csv(path_m5+"calendar.csv")
prices = pd.read_csv(path_m5+"sell_prices.csv")
sales = pd.read_csv(path_m5+"sales_train_evaluation.csv")

In [3]:
calendar = calendar.rename(columns={"d":"day"})

In [ ]:
calendar = encode_categorical(
    calendar, ["event_name_1", "event_type_1", "event_name_2", "event_type_2"]
).pipe(reduce_mem_usage)

sales = encode_categorical(
    sales, ["item_id", "dept_id", "cat_id", "store_id", "state_id"],
).pipe(reduce_mem_usage)

prices = encode_categorical(prices, ["item_id", "store_id"]).pipe(
    reduce_mem_usage
)

Mem. usage decreased to  0.07 Mb (66.9% reduction)


In [ ]:
prices_dates = pd.merge(prices, calendar[["wm_yr_wk", "date", "day", "weekday", "wday", "month"]],
                  on="wm_yr_wk", how="left")

In [ ]:
sales_ts = pd.melt(sales,
                 id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                 var_name = 'day', value_name = 'demand')

In [ ]:
del(sales)

In [ ]:
cols_group = ['item_id', 'store_id']
prices_dates['full_price'] = prices_dates.groupby(cols_group)['sell_price'].transform('max')

In [ ]:
prices_dates['discount_pct'] = (prices_dates['full_price']-prices_dates['sell_price'])/prices_dates['full_price']*100

In [ ]:
sales_ts = pd.merge(sales_ts, prices_dates[["day", "item_id", "store_id", "sell_price", "full_price", "discount_pct"]], 
                    on=["day", "item_id", "store_id"])
df_all = sales_ts.groupby("day").agg({"demand":"sum", 
                                      "sell_price":"mean", 
                                      "full_price":"mean", 
                                      "discount_pct":"mean"}).reset_index()
df_all = pd.merge(df_all, prices_dates[["day", "date", "weekday", "wday", "month"]], on="day")
df_all = df_all.rename(columns={"date":"ds", "demand":"y"})
df_all = df_all[["ds", "y"]]
df_all['ds'] = df_all['ds'].astype(np.datetime64)

In [ ]:
df_all.to_csv("sales_ms_all.csv", index=False)

In [ ]:
df_cat = sales_ts.groupby(["cat_id", "day"]).agg({"demand":"sum"}).reset_index()
df_cat = pd.merge(df_cat, calendar[["day", "date"]], on="day")
df_cat = df_cat.rename(columns={"date":"ds", "demand":"y"})
df_cat = df_cat[["ds", "cat_id", "y"]]
df_cat['ds'] = df_cat['ds'].astype(np.datetime64)

In [ ]:
df_cat.to_csv("sales_ms_cat.csv", index=False)